# Du scrapping à la reconnaissance d'entités nommées

Pour reconnaissance les noms des députés, je reprends le code utilisé pour scrapper les informations les concernant.

## Code de scrapping d'AGODA

In [1]:
def scrap_AN_AGODA(debut, fin):
    import requests
    import bs4 as bs
    import pandas as pd
    debut = int(debut)
    fin = int(fin + 1)
    legislatures = range(debut, fin)
    url = "http://www2.assemblee-nationale.fr/sycomore/liste/(legislature)/"
    df_complet = pd.DataFrame(columns=["id_depute", "nom", "prenom", "lien","naissance_j", "naissance_m", "naissance_a", "deces_j", "deces_m", "deces_a" ])

    for leg in legislatures:
        mois = {"janvier": "1", "février": "2", "mars": "3", "avril": "4", "mai": "5", "juin": "6",\
                "juillet": "7", "août": "7", "septembre": "9", "octobre": "10", "novembre": "11", "décembre": "12"}
        r = requests.get(url + str(leg))
        pageWeb = r.text
        localisation_depart = pageWeb.find("<table class=\"sycomore\">")
        localisation_fin = pageWeb.find("</table>")
        tableau_seul = pageWeb[localisation_depart:localisation_fin]
        soup = bs.BeautifulSoup(tableau_seul, 'lxml')
        all_tr = list(soup.find_all("tr"))
        all_tr_data = all_tr[1:]
        
        nom = []
        prenom = []
        lien = []
        id_depute = []
        date_naissance = []
        date_deces = []
        
        for data in all_tr_data:
            
            id_depute.append(int(data.find('a').get('href').replace("/sycomore/fiche/(num_dept)/", "")))
            nom.append(data.td['data-sort'])
            prenom.append(data.td.get_text().replace(data.find('strong').string, "").strip())
            lien.append("http://www2.assemblee-nationale.fr" + data.find('a').get('href'))
            date_naissance.append(data.find_all('td')[1].string)
            date_deces.append(data.find_all('td')[2].string)
    
        data_dic = {"id_depute": id_depute, 
                    "nom": nom, 
                    "prenom": prenom, 
                    "lien": lien, 
                    "date_naissance": date_naissance,
                    "date_deces": date_deces} 

        df = pd.DataFrame(data_dic)
        df[["date_naissance", "naissance_m", "naissance_a"]] = df["date_naissance"].str.split(expand=True).replace("1er", "1")
        df[["date_deces", "deces_m", "deces_a"]] = df["date_deces"].str.split(expand=True).replace("1er", "1")
        df.rename(columns = {"date_naissance": "naissance_j", "date_deces": "deces_j"}, inplace=True)
        df = df[["id_depute", "nom", "prenom", "lien","naissance_j", "naissance_m", "naissance_a",\
                 "deces_j", "deces_m", "deces_a" ]]
       
        df["naissance_m"] = df["naissance_m"].map(mois, na_action = 'ignore')
        df["deces_m"] = df["deces_m"].map(mois, na_action = 'ignore')
# On enregistre localement le DF contenant la liste des députés de la législature | changement personnel
        chemin_fichier_csv = "/Users/charlielezin/Desktop/test_scrapping_députés/scrap_AGODA.csv" 
        df.to_csv(chemin_fichier_csv, sep=",", index=False)
# On insère la liste des députés de la législature dans le DF df_complet en supprimant les doublons
        df_complet = pd.concat([df_complet, df], ignore_index=True).drop_duplicates()
    
    df_complet.to_csv("legislature" + str(debut) + "_" + str(fin - 1))
    return df_complet ## ajout de ligne

In [2]:
df_general = scrap_AN_AGODA(47, 47)

In [3]:
df_general

,id_depute,nom,prenom,lien,naissance_j,naissance_m,naissance_a,deces_j,deces_m,deces_a
0,19,Abelin,"Pierre, Louis, Ernest, Armand",http://www2.assemblee-nationale.fr/sycomore/fi...,16,5,1909,23,5,1977
1,79,Alduy,Paul,http://www2.assemblee-nationale.fr/sycomore/fi...,4,10,1914,23,1,2006
2,94,Alliot,"Edouard, Denis, Marie, Joseph",http://www2.assemblee-nationale.fr/sycomore/fi...,26,2,1900,16,11,1981
3,97,Alloin,"Louis, Marie",http://www2.assemblee-nationale.fr/sycomore/fi...,3,7,1906,28,12,1978
4,132,André,Pierre,http://www2.assemblee-nationale.fr/sycomore/fi...,27,10,1903,9,4,1984
...,...,...,...,...,...,...,...,...,...,...
610,7235,Villard,Jean,http://www2.assemblee-nationale.fr/sycomore/fi...,24,2,1907,10,12,1986
611,7237,Villon-Ginsburger,Pierre,http://www2.assemblee-nationale.fr/sycomore/fi...,27,7,1901,6,11,1980
612,7259,Vitter,"Pierre, Joseph, Louis, Léon",http://www2.assemblee-nationale.fr/sycomore/fi...,29,10,1913,26,10,1995
613,7243,Vuillien,André,http://www2.assemblee-nationale.fr/sycomore/fi...,9,2,1924,6,5,2016


## Mon scrap, adapté d'AGODA

In [4]:
def mon_scrap_AN(debut, fin):
    import requests
    import bs4 as bs
    import pandas as pd
    
    debut = int(debut)
    fin = int(fin + 1)
    legislatures = range(debut, fin)
    url = "https://www2.assemblee-nationale.fr/sycomore/liste/(legislature)/"
    df_complet = pd.DataFrame(columns=["lien", "groupe_parlementaire", "departement"])

    for leg in legislatures:
        r = requests.get(url + str(leg) + "/(alpha)/true")
        pageWeb = r.text
        localisation_depart = pageWeb.find("<article>")
        localisation_fin = pageWeb.find("</article>")
        tableau_seul = pageWeb[localisation_depart:localisation_fin]
        soup = bs.BeautifulSoup(tableau_seul, 'lxml')
        all_li = list(soup.find_all("li"))
        all_li_data = all_li[1:]

        lien = []
        groupe_parlementaire = []
        departement = []
        

        for data in all_li_data:
            a_tag = data.find("a")
            lien.append("http://www2.assemblee-nationale.fr" + a_tag.get('href'))

            # Extraction du groupe parlementaire du député
            groupe_element = data.find("span", class_="block bottommargin")
            if groupe_element is not None:
                groupe_i_element = groupe_element.find("i")
                groupe_text = groupe_i_element.text.strip() if groupe_i_element is not None else ""
                groupe_text = groupe_text.replace("(", "").replace(")", "")
                groupe_parlementaire.append(groupe_text)
            else:
                groupe_parlementaire.append("")

            # Extraction du département du député
            departement_element = data.find("span", class_="block bottommargin")
            if departement_element is not None:
                departement_span_elements = departement_element.find_all("span")
                if len(departement_span_elements) >= 2:
                    departement_text = departement_span_elements[1].text.strip().replace(" - ", "")
                    departement_text = departement_text.replace("(", "").replace(")", "")
                    departement.append(departement_text)
                else:
                    departement.append("")
            else:
                departement.append("")
                                   
        data_dic = {
                    "lien": lien,
                    "groupe_parlementaire": groupe_parlementaire,
                    "departement": departement
                    
                }

        df = pd.DataFrame(data_dic)
        df = df[["lien", "groupe_parlementaire", "departement"]]
        df[['departement', 'groupe_parlementaire']] = df['groupe_parlementaire'].str.split(' - ', 1, expand=True)
        df_complet = df_complet.append(df, ignore_index=True)

# On enregistre localement le DF contenant la liste des députés de la législature | changement personnel
        chemin_fichier_csv = "/Users/charlielezin/Desktop/test_scrapping_députés/scrap_groupe_dep2.csv" 
        df.to_csv(chemin_fichier_csv, sep=",", index=False)
# On insère la liste des députés de la législature dans le DF df_complet en supprimant les doublons
        df_complet = pd.concat([df_complet, df], ignore_index=True).drop_duplicates()

    df_complet.to_csv("legislature" + str(debut) + "_" + str(fin - 1) + ".csv", sep=",", index=False)
    return df_complet ## ajout de ligne

In [5]:
df_groupe_dep = mon_scrap_AN(47, 47)

/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_45943/3622332321.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_complet = df_complet.append(df, ignore_index=True)


In [6]:
df_groupe_dep

,lien,groupe_parlementaire,departement
0,http://www2.assemblee-nationale.fr/sycomore/fi...,Socialiste,Pyrénées-Orientales
1,http://www2.assemblee-nationale.fr/sycomore/fi...,Indépendants et paysans d'action sociale,Aisne
2,http://www2.assemblee-nationale.fr/sycomore/fi...,Union et fraternité francaise,Rhône
3,http://www2.assemblee-nationale.fr/sycomore/fi...,Indépendants et paysans d'action sociale,Meurthe-et-Moselle
4,http://www2.assemblee-nationale.fr/sycomore/fi...,Républicain radical et radical-socialiste,Vienne
...,...,...,...
609,http://www2.assemblee-nationale.fr/sycomore/fi...,Mouvement républicain populaire,Rhône
610,http://www2.assemblee-nationale.fr/sycomore/fi...,Communiste,Allier
611,http://www2.assemblee-nationale.fr/sycomore/fi...,Paysan,Haute-Saône
612,http://www2.assemblee-nationale.fr/sycomore/fi...,Communiste,Saône-et-Loire


In [7]:
import pandas as pd
# Ajout des informations du député Abelin
nouvelle_ligne = pd.Series({'lien': 'http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/19/(legislature)/47', 
                            'groupe_parlementaire': 'Mouvement républicain populaire', 'departement': 'Vienne'})

# Utiliser la méthode concat pour ajouter la nouvelle ligne en tant que première ligne
df_groupe_dep = pd.concat([nouvelle_ligne.to_frame().transpose(), df_groupe_dep], ignore_index=True)

In [8]:
df_groupe_dep ## 615 députés

,lien,groupe_parlementaire,departement
0,http://www2.assemblee-nationale.fr/sycomore/fi...,Mouvement républicain populaire,Vienne
1,http://www2.assemblee-nationale.fr/sycomore/fi...,Socialiste,Pyrénées-Orientales
2,http://www2.assemblee-nationale.fr/sycomore/fi...,Indépendants et paysans d'action sociale,Aisne
3,http://www2.assemblee-nationale.fr/sycomore/fi...,Union et fraternité francaise,Rhône
4,http://www2.assemblee-nationale.fr/sycomore/fi...,Indépendants et paysans d'action sociale,Meurthe-et-Moselle
...,...,...,...
610,http://www2.assemblee-nationale.fr/sycomore/fi...,Mouvement républicain populaire,Rhône
611,http://www2.assemblee-nationale.fr/sycomore/fi...,Communiste,Allier
612,http://www2.assemblee-nationale.fr/sycomore/fi...,Paysan,Haute-Saône
613,http://www2.assemblee-nationale.fr/sycomore/fi...,Communiste,Saône-et-Loire


In [9]:
# Je supprime la colonne lien, qui m'a servi à vérifier si les informations revenaient dans le même ordre
# à partir de mon code et avec celui d'AGODA
df_groupe_dep = df_groupe_dep.drop('lien', axis=1)
df_groupe_dep

,groupe_parlementaire,departement
0,Mouvement républicain populaire,Vienne
1,Socialiste,Pyrénées-Orientales
2,Indépendants et paysans d'action sociale,Aisne
3,Union et fraternité francaise,Rhône
4,Indépendants et paysans d'action sociale,Meurthe-et-Moselle
...,...,...
610,Mouvement républicain populaire,Rhône
611,Communiste,Allier
612,Paysan,Haute-Saône
613,Communiste,Saône-et-Loire


In [10]:
df_final = pd.concat([df_general, df_groupe_dep], axis=1) ## je concatène les deux dataframe

In [11]:
df_final

,id_depute,nom,prenom,lien,naissance_j,naissance_m,naissance_a,deces_j,deces_m,deces_a,groupe_parlementaire,departement
0,19,Abelin,"Pierre, Louis, Ernest, Armand",http://www2.assemblee-nationale.fr/sycomore/fi...,16,5,1909,23,5,1977,Mouvement républicain populaire,Vienne
1,79,Alduy,Paul,http://www2.assemblee-nationale.fr/sycomore/fi...,4,10,1914,23,1,2006,Socialiste,Pyrénées-Orientales
2,94,Alliot,"Edouard, Denis, Marie, Joseph",http://www2.assemblee-nationale.fr/sycomore/fi...,26,2,1900,16,11,1981,Indépendants et paysans d'action sociale,Aisne
3,97,Alloin,"Louis, Marie",http://www2.assemblee-nationale.fr/sycomore/fi...,3,7,1906,28,12,1978,Union et fraternité francaise,Rhône
4,132,André,Pierre,http://www2.assemblee-nationale.fr/sycomore/fi...,27,10,1903,9,4,1984,Indépendants et paysans d'action sociale,Meurthe-et-Moselle
...,...,...,...,...,...,...,...,...,...,...,...,...
610,7235,Villard,Jean,http://www2.assemblee-nationale.fr/sycomore/fi...,24,2,1907,10,12,1986,Mouvement républicain populaire,Rhône
611,7237,Villon-Ginsburger,Pierre,http://www2.assemblee-nationale.fr/sycomore/fi...,27,7,1901,6,11,1980,Communiste,Allier
612,7259,Vitter,"Pierre, Joseph, Louis, Léon",http://www2.assemblee-nationale.fr/sycomore/fi...,29,10,1913,26,10,1995,Paysan,Haute-Saône
613,7243,Vuillien,André,http://www2.assemblee-nationale.fr/sycomore/fi...,9,2,1924,6,5,2016,Communiste,Saône-et-Loire


In [12]:
import pandas as pd

# J'exporte le DataFrame dans un fichier CSV
df_final.to_csv('informations_députés.csv', index=False)

In [13]:
# Compter le nombre d'occurrences dans la colonne 'groupe_parlementaire'
occurrences = df_final['groupe_parlementaire'].value_counts()

# Afficher les résultats
print(occurrences)

Communiste                                                          145
Socialiste                                                          101
Indépendants et paysans d'action sociale                             88
Mouvement républicain populaire                                      74
Républicain radical et radical-socialiste                            60
Union et fraternité francaise                                        38
Républicains sociaux                                                 24
Union démocratique et socialiste de la Résistance et du RDA          21
Union et fraternité française                                        14
Rassemblement des gauches républicaines et du centre républicain     14
Paysan                                                               13
Indépendants d'outre-mer                                             10
Républicain progressiste                                              5
Non inscrit                                                     

In [14]:
groupes_parlementaires = ['Communiste', 'Socialiste', 'Républicain radical et radical-socialiste', 'Union démocratique et socialiste de la Résistance et du RDA']

df_gauche = df_final[df_final['groupe_parlementaire'].isin(groupes_parlementaires)]

In [15]:
df_gauche

,id_depute,nom,prenom,lien,naissance_j,naissance_m,naissance_a,deces_j,deces_m,deces_a,groupe_parlementaire,departement
1,79,Alduy,Paul,http://www2.assemblee-nationale.fr/sycomore/fi...,4,10,1914,23,1,2006,Socialiste,Pyrénées-Orientales
5,128,André,"Adrien, Baptiste, Honoré",http://www2.assemblee-nationale.fr/sycomore/fi...,29,5,1884,22,4,1965,Républicain radical et radical-socialiste,Vienne
7,153,Ansart,Gustave,http://www2.assemblee-nationale.fr/sycomore/fi...,5,3,1923,20,9,1990,Communiste,Nord
10,165,Anxionnaz,"Paul, Marius",http://www2.assemblee-nationale.fr/sycomore/fi...,31,12,1902,20,2,1997,Républicain radical et radical-socialiste,Marne
12,176,Arbeltier,"René, Albert",http://www2.assemblee-nationale.fr/sycomore/fi...,23,1,1897,10,9,1979,Socialiste,Seine-et-Marne
...,...,...,...,...,...,...,...,...,...,...,...,...
602,7221,Vergès,Paul,http://www2.assemblee-nationale.fr/sycomore/fi...,5,3,1925,12,11,2016,Communiste,La Réunion
603,7222,Vermeersch-Thorez,Jeannette,http://www2.assemblee-nationale.fr/sycomore/fi...,26,11,1910,5,11,2001,Communiste,Seine
604,7226,Véry-Hermence,Emmanuel,http://www2.assemblee-nationale.fr/sycomore/fi...,31,3,1904,19,6,1966,Socialiste,Martinique
611,7237,Villon-Ginsburger,Pierre,http://www2.assemblee-nationale.fr/sycomore/fi...,27,7,1901,6,11,1980,Communiste,Allier


In [16]:
df_gauche.to_csv('deputes_gauche.csv', index=False)

## Reconnaissance d'entités nommées

#### Création d'un dictionnaire à partir du df_gauche

In [17]:
# Conversion du DataFrame en dictionnaire
deputes_gauche_dict1 = df_gauche[['nom', 'prenom']].to_dict(orient='records')

print(deputes_gauche_dict1)

[{'nom': 'Alduy', 'prenom': 'Paul'}, {'nom': 'André', 'prenom': 'Adrien, Baptiste, Honoré'}, {'nom': 'Ansart', 'prenom': 'Gustave'}, {'nom': 'Anxionnaz', 'prenom': 'Paul, Marius'}, {'nom': 'Arbeltier', 'prenom': 'René, Albert'}, {'nom': 'Arnal', 'prenom': 'Franck'}, {'nom': 'Arrighi', 'prenom': 'Pascal'}, {'nom': 'Auban', 'prenom': 'Achille, Joseph'}, {'nom': 'Babet', 'prenom': 'Raphaël, Jean-Baptiste, Benjamin'}, {'nom': 'Badie', 'prenom': 'Vincent, Henri'}, {'nom': 'Ballanger', 'prenom': 'Robert, Victor, Marcel, François'}, {'nom': 'Barbot', 'prenom': 'Marcel'}, {'nom': 'Barel', 'prenom': 'Joseph dit Virgile'}, {'nom': 'Barry', 'prenom': 'Diawadou'}, {'nom': 'Barthélémy', 'prenom': 'André, Jules, François'}, {'nom': 'Bartolini', 'prenom': 'Jean, Baptiste'}, {'nom': 'Baurens', 'prenom': 'Alexandre, Sylvain, Justin'}, {'nom': 'Baylet', 'prenom': 'Jean, André, Bernard'}, {'nom': 'Bégouin', 'prenom': 'Lucien, Louis, Marie'}, {'nom': 'Béné', 'prenom': 'Maurice'}, {'nom': 'Benoist', 'preno

Je veux conserver uniquement le premier prénom et le premier nom des députés. 

In [50]:
# Conversion du DataFrame en dictionnaire avec les premiers prénoms et noms
deputes_gauche_dict2 = df_gauche.apply(lambda row: {'prenom': row['prenom'].split(',')[0].strip(),
                                     'nom': row['nom'].split(' ')[0].strip()}, axis=1).tolist()

print(deputes_gauche_dict2)

[{'prenom': 'Paul', 'nom': 'Alduy'}, {'prenom': 'Adrien', 'nom': 'André'}, {'prenom': 'Gustave', 'nom': 'Ansart'}, {'prenom': 'Paul', 'nom': 'Anxionnaz'}, {'prenom': 'René', 'nom': 'Arbeltier'}, {'prenom': 'Franck', 'nom': 'Arnal'}, {'prenom': 'Pascal', 'nom': 'Arrighi'}, {'prenom': 'Achille', 'nom': 'Auban'}, {'prenom': 'Raphaël', 'nom': 'Babet'}, {'prenom': 'Vincent', 'nom': 'Badie'}, {'prenom': 'Robert', 'nom': 'Ballanger'}, {'prenom': 'Marcel', 'nom': 'Barbot'}, {'prenom': 'Joseph dit Virgile', 'nom': 'Barel'}, {'prenom': 'Diawadou', 'nom': 'Barry'}, {'prenom': 'André', 'nom': 'Barthélémy'}, {'prenom': 'Jean', 'nom': 'Bartolini'}, {'prenom': 'Alexandre', 'nom': 'Baurens'}, {'prenom': 'Jean', 'nom': 'Baylet'}, {'prenom': 'Lucien', 'nom': 'Bégouin'}, {'prenom': 'Maurice', 'nom': 'Béné'}, {'prenom': 'Charles', 'nom': 'Benoist'}, {'prenom': 'Alcide', 'nom': 'Benoit'}, {'prenom': 'Alix', 'nom': 'Berthet'}, {'prenom': 'Pierre', 'nom': 'Besset'}, {'prenom': 'Paul', 'nom': 'Billat'}, {'pre

Cela n'allait pas, car certains députés ont un nom à plusieurs mots, notamment ceux qui ont une particule (de) ou originaires de Bretagne (le)

In [51]:
def prepare_nom(nom):
    mots = nom.split(' ')
    # Si le premier mot du nom est "Le" ou "de", retournez le premier et le deuxième mot
    if mots[0] in ["Le", "de"]:
        return " ".join(mots[:2])
    return mots[0]

deputes_gauche_dict3 = df_gauche.apply(lambda row: {'prenom': row['prenom'].split(',')[0].strip(),
                                                    'nom': prepare_nom(row['nom'])}, axis=1).tolist()

print(deputes_gauche_dict3)

[{'prenom': 'Paul', 'nom': 'Alduy'}, {'prenom': 'Adrien', 'nom': 'André'}, {'prenom': 'Gustave', 'nom': 'Ansart'}, {'prenom': 'Paul', 'nom': 'Anxionnaz'}, {'prenom': 'René', 'nom': 'Arbeltier'}, {'prenom': 'Franck', 'nom': 'Arnal'}, {'prenom': 'Pascal', 'nom': 'Arrighi'}, {'prenom': 'Achille', 'nom': 'Auban'}, {'prenom': 'Raphaël', 'nom': 'Babet'}, {'prenom': 'Vincent', 'nom': 'Badie'}, {'prenom': 'Robert', 'nom': 'Ballanger'}, {'prenom': 'Marcel', 'nom': 'Barbot'}, {'prenom': 'Joseph dit Virgile', 'nom': 'Barel'}, {'prenom': 'Diawadou', 'nom': 'Barry'}, {'prenom': 'André', 'nom': 'Barthélémy'}, {'prenom': 'Jean', 'nom': 'Bartolini'}, {'prenom': 'Alexandre', 'nom': 'Baurens'}, {'prenom': 'Jean', 'nom': 'Baylet'}, {'prenom': 'Lucien', 'nom': 'Bégouin'}, {'prenom': 'Maurice', 'nom': 'Béné'}, {'prenom': 'Charles', 'nom': 'Benoist'}, {'prenom': 'Alcide', 'nom': 'Benoit'}, {'prenom': 'Alix', 'nom': 'Berthet'}, {'prenom': 'Pierre', 'nom': 'Besset'}, {'prenom': 'Paul', 'nom': 'Billat'}, {'pre

In [52]:
df_gauche_corrigé = pd.DataFrame(deputes_gauche_dict3)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
print(df_gauche_corrigé) ## Reste Raymond dit Gérard

                              prenom                 nom
0                               Paul               Alduy
1                             Adrien               André
2                            Gustave              Ansart
3                               Paul           Anxionnaz
4                               René           Arbeltier
5                             Franck               Arnal
6                             Pascal             Arrighi
7                            Achille               Auban
8                            Raphaël               Babet
9                            Vincent               Badie
10                            Robert           Ballanger
11                            Marcel              Barbot
12                Joseph dit Virgile               Barel
13                          Diawadou               Barry
14                             André          Barthélémy
15                              Jean           Bartolini
16                         Alex

In [53]:
for dict in deputes_gauche_dict3:
    if dict["prenom"] == "Raymond dit Gérard":
        dict["prenom"] = "Gérard"

In [54]:
df_gauche_corrigé2 = pd.DataFrame(deputes_gauche_dict3)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
print(df_gauche_corrigé2) 

                              prenom                 nom
0                               Paul               Alduy
1                             Adrien               André
2                            Gustave              Ansart
3                               Paul           Anxionnaz
4                               René           Arbeltier
5                             Franck               Arnal
6                             Pascal             Arrighi
7                            Achille               Auban
8                            Raphaël               Babet
9                            Vincent               Badie
10                            Robert           Ballanger
11                            Marcel              Barbot
12                Joseph dit Virgile               Barel
13                          Diawadou               Barry
14                             André          Barthélémy
15                              Jean           Bartolini
16                         Alex

#### Conversion des pdf en txt

In [19]:
#pip install pdfminer.six

#### Extraction pdf 1958_1

In [20]:
import os
from pdfminer.high_level import extract_text

def convert_pdf_to_txt(folder_path, output_filename):
    output_file = open(output_filename, "w")
    for foldername, subfolders, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.pdf'):
                path_to_pdf = os.path.join(foldername, filename)
                text = extract_text(path_to_pdf)
                output_file.write(text + "\n")
    output_file.close()

convert_pdf_to_txt('/Volumes/Elements/JO_débats_1956-1958/JO_Débats_1958_1/', 'débats1958_1.txt') 
# on appelle un répertoire de départ et un fichier d'arrivée

In [55]:
deputes_gauche_dict3

[{'prenom': 'Paul', 'nom': 'Alduy'},
 {'prenom': 'Adrien', 'nom': 'André'},
 {'prenom': 'Gustave', 'nom': 'Ansart'},
 {'prenom': 'Paul', 'nom': 'Anxionnaz'},
 {'prenom': 'René', 'nom': 'Arbeltier'},
 {'prenom': 'Franck', 'nom': 'Arnal'},
 {'prenom': 'Pascal', 'nom': 'Arrighi'},
 {'prenom': 'Achille', 'nom': 'Auban'},
 {'prenom': 'Raphaël', 'nom': 'Babet'},
 {'prenom': 'Vincent', 'nom': 'Badie'},
 {'prenom': 'Robert', 'nom': 'Ballanger'},
 {'prenom': 'Marcel', 'nom': 'Barbot'},
 {'prenom': 'Joseph dit Virgile', 'nom': 'Barel'},
 {'prenom': 'Diawadou', 'nom': 'Barry'},
 {'prenom': 'André', 'nom': 'Barthélémy'},
 {'prenom': 'Jean', 'nom': 'Bartolini'},
 {'prenom': 'Alexandre', 'nom': 'Baurens'},
 {'prenom': 'Jean', 'nom': 'Baylet'},
 {'prenom': 'Lucien', 'nom': 'Bégouin'},
 {'prenom': 'Maurice', 'nom': 'Béné'},
 {'prenom': 'Charles', 'nom': 'Benoist'},
 {'prenom': 'Alcide', 'nom': 'Benoit'},
 {'prenom': 'Alix', 'nom': 'Berthet'},
 {'prenom': 'Pierre', 'nom': 'Besset'},
 {'prenom': 'Paul',

In [57]:
import re

# Orateurs issus de notre dictionnaire
orateurs = deputes_gauche_dict3

# Préparation des noms complets pour la recherche regex
noms_complets = ["{} {}".format(orateur['prenom'], orateur['nom']) for orateur in orateurs]

# Lecture du fichier
with open('débats1958_1.txt', 'r') as fichier:
    texte = fichier.read()

# Compte des occurences pour chaque député
for nom_complet in noms_complets:
    occurences = len(re.findall(re.escape(nom_complet), texte))
    print(f"Le député {nom_complet} est mentionné {occurences} fois.")

Le député Paul Alduy est mentionné 0 fois.
Le député Adrien André est mentionné 0 fois.
Le député Gustave Ansart est mentionné 0 fois.
Le député Paul Anxionnaz est mentionné 0 fois.
Le député René Arbeltier est mentionné 0 fois.
Le député Franck Arnal est mentionné 0 fois.
Le député Pascal Arrighi est mentionné 0 fois.
Le député Achille Auban est mentionné 0 fois.
Le député Raphaël Babet est mentionné 0 fois.
Le député Vincent Badie est mentionné 0 fois.
Le député Robert Ballanger est mentionné 0 fois.
Le député Marcel Barbot est mentionné 0 fois.
Le député Joseph dit Virgile Barel est mentionné 0 fois.
Le député Diawadou Barry est mentionné 0 fois.
Le député André Barthélémy est mentionné 0 fois.
Le député Jean Bartolini est mentionné 0 fois.
Le député Alexandre Baurens est mentionné 0 fois.
Le député Jean Baylet est mentionné 0 fois.
Le député Lucien Bégouin est mentionné 0 fois.
Le député Maurice Béné est mentionné 0 fois.
Le député Charles Benoist est mentionné 0 fois.
Le député Al

Le député Marcel Levindrey est mentionné 0 fois.
Le député Gabriel Lisette est mentionné 0 fois.
Le député Jean Llante est mentionné 0 fois.
Le député Kléber Loustau est mentionné 0 fois.
Le député Adrien Mabrut est mentionné 0 fois.
Le député Pierre Mailhé est mentionné 0 fois.
Le député Alfred Malleret-Joinville est mentionné 0 fois.
Le député Robert Manceau est mentionné 0 fois.
Le député André Mancey est mentionné 1 fois.
Le député Hervé Mao est mentionné 0 fois.
Le député Charles Margueritte est mentionné 0 fois.
Le député René Mariat est mentionné 0 fois.
Le député André Marie est mentionné 0 fois.
Le député Fernand Marin est mentionné 0 fois.
Le député André Maroselli est mentionné 0 fois.
Le député Georges Marrane est mentionné 0 fois.
Le député Henri Martel est mentionné 0 fois.
Le député Gilbert Martin est mentionné 0 fois.
Le député Madeleine Marzin est mentionné 0 fois.
Le député Jean Masse est mentionné 0 fois.
Le député Jean Masson est mentionné 0 fois.
Le député Albert M

Cela ne va pas, quasiment tous les députés ne sont pas mentionnés, ou très rarement.

#### Pour compter le nombre de caractères 

In [23]:
filename = '/Users/charlielezin/Desktop/REN/REN_1958_1/débats1958_1.txt'  


file = open(filename, 'r')
content = file.read()
character_count = len(content)
file.close()

print("Le fichier", filename, "contient", character_count, "caractères.")

Le fichier /Users/charlielezin/Desktop/REN/REN_1958_1/débats1958_1.txt contient 10418584 caractères.


#### Tentative d'extraction des prises de paroles

In [1]:
import re

def extraction_discours(texte, prenoms_noms, noms):
    prenoms_noms_regex = '|'.join(prenoms_noms)
    noms_regex = '|'.join(noms)

    modele = r"(M\.|Mme|Mlle)\s+(" + prenoms_noms_regex + "|" + noms_regex + r")\.(.*?)(?=(M\.|Mme|Mlle|\())"
    
    discours = re.findall(modele, texte, re.DOTALL)
    
    return discours

def traitement_file(input_fichier, orateurs):
    with open(input_fichier, 'r') as file:
        texte = file.read()

    noms = [orateur['nom'] for orateur in orateurs]
    prenoms_noms = [f"{orateur['prenom']} {orateur['nom']}" for orateur in orateurs]

    discours = extraction_discours(texte, prenoms_noms, noms)

    for discours in discours:
        print(discours)

# Appelez la fonction avec le nom de votre fichier en entrée et la liste de dictionnaires
orateurs = deputes_gauche_dict3
traitement_file('débats1958_1.txt', orateurs)

NameError: name 'deputes_gauche_dict3' is not defined

In [26]:
import re
import csv

def extraire_discours(texte, noms_complets, noms_familiaux):
    noms_complets_regex = '|'.join(noms_complets)
    noms_familiaux_regex = '|'.join(noms_familiaux)

    motif = r"(M\.|Mme|Mlle)\s+(" + noms_complets_regex + "|" + noms_familiaux_regex + r")\.(.*?)(?=(M\.|Mme|Mlle|\())"
    
    discours = re.findall(motif, texte, re.DOTALL)
    
    # Pour conserver les discours de plus d'une ligne
    discours = [d for d in discours if '\n' in d[2]]
    
    return discours

def traiter_fichier(nom_fichier_input, orateurs):
    with open(nom_fichier_input, 'r') as fichier:
        texte = fichier.read()

    noms_familiaux = [orateur['nom'] for orateur in orateurs]
    noms_complets = [f"{orateur['prenom']} {orateur['nom']}" for orateur in orateurs]

    discours = extraire_discours(texte, noms_complets, noms_familiaux)
    
    with open('discoursGauche_1958_1.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Titre", "Nom", "Discours"])
        for d in discours:
            writer.writerow([d[0], d[1], d[2].replace('\n', ' ')])

orateurs = deputes_gauche_dict3

traiter_fichier('débats1958_1.txt', orateurs)

In [32]:
import re
import pandas as pd

def supprime_saut_de_ligne_avant_parentheses(texte):
    return re.sub(r"\n\(", " (", texte)

def extrait_discours(texte, noms_complets, noms_de_famille):
    noms_complets_regex = '|'.join(map(re.escape, noms_complets))
    noms_de_famille_regex = '|'.join(map(re.escape, noms_de_famille))

    motif = r"(M\.|Mme|Mlle)\s+((?:" + noms_complets_regex + r")|(?:" + noms_de_famille_regex + r"))\.(.*?)(?=(M\.|Mme|Mlle)(?:(?!\s+(?:" + noms_complets_regex + r"|(?:" + noms_de_famille_regex + r")))|\())"
    
    discours = re.findall(motif, texte, re.DOTALL)
    
    # Conserver uniquement les groupes pertinents pour chaque match
    discours = [(match[0], match[1], match[2]) for match in discours]
    
    return discours

def traite_fichier(fichier_entree, orateurs):
    with open(fichier_entree, 'r') as fichier:
        texte = fichier.read()

    texte = supprime_saut_de_ligne_avant_parentheses(texte)

    noms_de_famille = [orateur['nom'] for orateur in orateurs]
    noms_complets = ["{} {}".format(orateur['prenom'], orateur['nom']) for orateur in orateurs]

    discours = extrait_discours(texte, noms_complets, noms_de_famille)

    df_discours = pd.DataFrame(discours, columns=['titre', 'orateur', 'discours'])

    # Ecrit le DataFrame dans un fichier CSV
    df_discours.to_csv('discoursGauche_1958_1_v2.csv', index=False)

    return df_discours

orateurs = deputes_gauche_dict2

df = traite_fichier('débats1958_1.txt', orateurs)

# Affiche le DataFrame
print(df)


    titre    orateur                                           discours
0      M.      Darou                                               \n\n
1      M.     Dagain   \n\n10 \n\nASSEMBLEE  NATIONALE  —  SEANCE  ...
2      M.     Besset   \nAdoption,  au  scrutin,  des  propositions ...
3      M.  Ballanger   \nAdoption,  au  scrutin. \nExplications  de ...
4      M.    Bourbon    —  Retrait. \nDemande  de  disjonction  de  ...
..    ...        ...                                                ...
383    M.    Ségelle   \n\n• •• \n\nQUESTIONS  ECRITES \nREMISES  A...
384   Mme    Degrond   \nDeixonne. \nDejean. \nDélabré. \nDelachenal...
385   Mme  Lempereur   \nLéotard  (de). \nLe  Slrat. \nLevindrev. \n...
386  Mlle     Marzin   \nMaton. \nMercier  (André),  Oise \nMerle. \...
387    M.     Gozard          (Applaudissements \n\ngauehe.) \n\ni \n\n

[388 rows x 3 columns]
